## Information

[facenet-pytorch](https://github.com/timesler/facenet-pytorch)

[face-tracking](https://github.com/timesler/facenet-pytorch/blob/master/examples/face_tracking.ipynb)

[use personal data](https://github.com/timesler/facenet-pytorch/blob/master/examples/infer.ipynb)

## Packages

In [24]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os

## Define run parameters 

In [ ]:
data_dir = 'data'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

In [ ]:
## Determine if an nvidia GPU is available¶

In [25]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


## Define MTCNN module

In [26]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

## Perfom MTCNN facial detection 

In [30]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn



## Define Inception Resnet V1 module¶ 

In [31]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

## Define optimizer, scheduler, dataset, and dataloader¶

In [32]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

## Define loss and evaluation functions

In [33]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

Face detected with probability: 0.999994
Face detected with probability: 0.999999
Face detected with probability: 1.000000
Face detected with probability: 0.999988
Face detected with probability: 1.000000
Face detected with probability: 0.998185
Face detected with probability: 0.999998
Face detected with probability: 0.999816
Face detected with probability: 0.999999
Face detected with probability: 0.999992
Face detected with probability: 0.999999
Face detected with probability: 0.999956
Face detected with probability: 0.999983
Face detected with probability: 1.000000
Face detected with probability: 0.999965
Face detected with probability: 1.000000
Face detected with probability: 0.999993
Face detected with probability: 0.999998
Face detected with probability: 0.999942
Face detected with probability: 0.998134
Face detected with probability: 0.999943
Face detected with probability: 0.999982
Face detected with probability: 0.999993
Face detected with probability: 0.999000
Face detected wi

## Train model

In [36]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor